# Malware Analysis & Triage Kit
This notebook performs the initial stages of immediate malware triage.

## How To
Take your malware specimen and drop it into the `dropbox` directory. The notebook will walk you through the stages of initial analysis.

At the end of this process, you will have a triage report in the `saved-specimens` diretory. This report includes findings from initial triage, including the defanged specimen in a password-proteced Zip file and static analysis artifacts.

# Imports and Setup

In [1]:
# Imports
from hashlib import *
import sys
import os
from getpass import getpass
from virus_total_apis import PublicApi as VirusTotalPublicApi
import json
from MalwareSample import *
from pprint import pprint
import os.path

### Check Dropbox and Saved-Specimens

In [2]:
MalwareSample.check_dir("dropbox")
MalwareSample.check_dir("saved-specimens")
empty = MalwareSample.is_dir_empty("dropbox")
if empty:
    print(r"  \\--> " + recc + "Put some samples in the dropbox!")

[*] Directory 'dropbox' exists.
[*] Directory 'saved-specimens' exists.


### Enumerate Samples in the Dropbox

In [3]:
samples=!ls dropbox/*
for s in samples:
    print(info + "Sample: " + s)

[*] Sample: dropbox/SampleNegative.txt
[*] Sample: dropbox/SamplePositive.txt


In [4]:
sample_obj = [MalwareSample(s) for s in samples]

### Create a Saved Specimen directory for the specimen(s)

In [5]:
for obj in sample_obj:
    saved_sample_name = MalwareSample.create_specimen_dirs(obj.sample_name)
    obj.saved_sample_name = saved_sample_name

### Defang Sample

In [6]:
for obj in sample_obj:
    sample_path = MalwareSample.move_and_defang(obj.sample_name, obj.saved_sample_name)
    obj.sample_path = sample_path

---

## File Hashes

### SHA256 Sum

In [7]:
for obj in sample_obj:
    hash = MalwareSample.get_sha256sum(obj.sample_path, obj.saved_sample_name)
    obj.sha256sum = hash
    print(info + obj.sample_name + ": " + obj.sha256sum)

[*] SampleNegative.txt: 5814318064326228845b4b2e8f4ae11d0d249475e2845e5eda05fb84bbb19bce
[*] SamplePositive.txt: 275a021bbfb6489e54d471899f7db9d1663fc695ec2fe2a2c4538aabf651fd0f


---

## String Analysis

### StringSifter
StringSifter is a FLARE developed tool that uses an ML model to rank a binary's strings by relevance to malware analysis.

In [8]:
length = int(input(recc + "Input your desired minimum string length [default is 4, 6-8 is recommended] > "))

[*] Input your desired minimum string length [default is 4, 6-8 is recommended] >  8


In [9]:
for obj in sample_obj:
    MalwareSample.pull_strings(length, obj.saved_sample_name, obj.sample_path)

[*] Written to outfile: saved-specimens/10-02-2021-213810_SampleNegative.txt/StringSifter-Out.log
[*] Written to outfile: saved-specimens/10-02-2021-213810_SamplePositive.txt/StringSifter-Out.log


## VT Analysis
Submit samples to Virus Total and generate a malicious confidence level.

In [10]:
VT_API_KEY = getpass("Enter VirusTotal API Key (blank if none): ")

Enter VirusTotal API Key (blank if none):  


In [11]:
if VT_API_KEY:
    vt = VirusTotalPublicApi(VT_API_KEY)
else:
    print(info + "No VT API Key. Skipping...")

[*] No VT API Key. Skipping...


In [12]:
if VT_API_KEY:
    for obj in sample_obj:
        print(info + obj.sample_name + ":")
        print(r"  \\--> " + info + "SHA256sum: " + obj.sha256sum)
        res = vt.get_file_report(obj.sha256sum)
        conf = malicious_confidence(res)
        print(r"  \\--> " + info + "Confidence level: " + str(conf))
        crit_level =  determine_criticality(conf)
        obj.criticality = crit_level
else:
    print(info + "No VT API Key. Skipping...")

[*] No VT API Key. Skipping...


## Zip and Password Protect

In [13]:
for obj in sample_obj:
    zip_file = MalwareSample.zip_and_password_protect(obj.sample_path, obj.saved_sample_name)
    MalwareSample.delete_unzipped_sample(obj.sample_path, zip_file)

---

### Debug Object Vars

In [14]:
for obj in sample_obj:
    pprint(vars(obj))

{'sample_name': 'SampleNegative.txt',
 'sample_path': 'saved-specimens/10-02-2021-213810_SampleNegative.txt/Malware.SampleNegative.txt.malz',
 'saved_sample_name': '10-02-2021-213810_SampleNegative.txt',
 'sha256sum': '5814318064326228845b4b2e8f4ae11d0d249475e2845e5eda05fb84bbb19bce'}
{'sample_name': 'SamplePositive.txt',
 'sample_path': 'saved-specimens/10-02-2021-213810_SamplePositive.txt/Malware.SamplePositive.txt.malz',
 'saved_sample_name': '10-02-2021-213810_SamplePositive.txt',
 'sha256sum': '275a021bbfb6489e54d471899f7db9d1663fc695ec2fe2a2c4538aabf651fd0f'}
